Importing Necessary Libraries.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN

Mounting drive for using the dataset.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/NLP/Assignment 2/dataset.txt", "r", encoding="utf-8") as myfile:
  myText = myfile.read()

Dataset is of a news article published by the Indian Express.

In [ ]:
myText

# Text Preprocessing

Creating a tokenizer object and finding the total unique words in the corpus.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([myText])
total_words = len(tokenizer.word_index) + 1

In [ ]:
tokenizer.word_index

Creating an input sequence based on the number provided by the tokenizer object.

In [ ]:
input_seq = []

for eachline in myText.split('\n'):
  # print(eachline)

  token_list = tokenizer.texts_to_sequences([eachline])[0]
  print(token_list)

  for i in range(1, len(token_list)):
    n_gram_seq = token_list[:i+1]
    # print(n_gram_seq)
    input_seq.append(n_gram_seq)
    # print(input_seq)

Finding the maximum sequence length and applying pre-padding to the input sentences with the help of maximum sequence length.

In [ ]:
max_seq_len = max([len(seq) for seq in input_seq])

input_sentences = np.array(pad_sequences(
    input_seq,
    maxlen=max_seq_len,
    padding='pre'
))

In [ ]:
input_sentences[1]

Dividing X and y such that such that current word(s) array is stored in X and the corresponding next word's value is stored in y.

X[1] shows the current and previous word’s, word index and the corresponding y[1] shows the next word’s, word index.

In [ ]:
X = input_sentences[:, :-1]
y = input_sentences[:, -1]

In [ ]:
X[1]

In [ ]:
y[1]

Converting y into a matrix where the value of a particular word is stored at that location (word index value) in a matrix of the size of total words in the corpus.

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))
y[1]

# Model Creation

Using LSTM

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_seq_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Accuracy turns out to be 99% using LSTM.

In [ ]:
model.fit(X, y, epochs=45, verbose=1)

In [ ]:
def generate_text(input_text, pred_next_word):
  for _ in range(pred_next_word):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')

    pred = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""

    for word, index in tokenizer.word_index.items():
      if index == pred:
        output_word = word
        break

    input_text += " " + output_word
  return input_text


In [ ]:
output = generate_text("Earlier this", 100)
print(output)